Engy-4350: Nuclear Reactor Engineering Spring 2019 UMass Lowell; Prof. V. F. de Almeida **03Feb2019**

# 02. Nuclear Data and Data Processing 
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\uvar}{\boldsymbol{u}}
  \newcommand{\fvar}{\boldsymbol{f}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Power}{\mathcal{P}}
$

---
## Table of Contents
* [Objectives](#obj)
* [Introduction](#intro)
* [Radiative capture cross section example](#xs1)
* [Beta decay example](#beta1)

---

## Objectives<a id="obj"></a>
+ Demonstrate how to obtain traceable nuclear data and have them available through the notebook for analysis and problem solving.

## Introduction<a id="intro"></a>

Many nuclear data sets are availale through the US [National Nuclear Data Center](https://www.nndc.bnl.gov/). Some important sites are as follows:

+ [Interactive chart of nuclides](https://www.nndc.bnl.gov/nudat2/).
+ [Nuclide energy level and $\gamma$ emission](https://www.nndc.bnl.gov/nudat2/indx_adopted.jsp).
+ [Nuclear wallet card](https://www.nndc.bnl.gov/nudat2/indx_sigma.jsp).
+ [Decay radiation search](https://www.nndc.bnl.gov/nudat2/indx_dec.jsp)
+ [Q-value calculator](https://www.nndc.bnl.gov/qcalc/) (difference of kinetic energy of products and reactant particles in a nuclear reaction, *i.e.* energy release).
+ [Atomic mass evaluator](http://amdc.impcas.ac.cn/web/masseval.html).
+ [Evaluated nuclear data file (ENDF)](https://www.nndc.bnl.gov/exfor/endf00.jsp).
+ [Cross sections](https://www.nndc.bnl.gov/sigma/index.jsp) for many nuclear reactions.

Similarly from [IAEA](https://www-nds.iaea.org/) Nuclear Data Services. Some important sites are as follows:

+ [Live chart of nuclides](https://www-nds.iaea.org/relnsd/vcharthtml/VChartHTML.html)
+ [Search engine](https://www-nds.iaea.org/relnsd/NdsEnsdf/QueryForm.html)

In addition from France's [LNHB](http://www.lnhb.fr/nuclear-data/nuclear-data-table/):
